# Kaggle Submission

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
import pickle
import numpy as np
np.random.seed(42)

In [2]:
# use pickle.load to load in these assets
with open('../assets/lasso.pkl', 'rb') as f:
    lasso = pickle.load(f)

#get the scaler for X_test_threshold...
with open('../assets/scaler.pkl', 'rb') as f:
    ss = pickle.load(f) 
    
with open('../assets/columns.pkl', 'rb') as f:
    model_columns = pickle.load(f)

In [3]:
# read in our test.csv as `kaggle`
kaggle = pd.read_csv('../datasets/test.csv', index_col = 'Id')

In [4]:
kaggle.head()

,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,...,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type
Id,,,,,,,,,,,,,,,,,,,,,
2658,902301120,190,RM,69.0,9142,Pave,Grvl,Reg,Lvl,AllPub,...,0,0,0,NaN,NaN,NaN,0,4,2006,WD
2718,905108090,90,RL,NaN,9662,Pave,NaN,IR1,Lvl,AllPub,...,0,0,0,NaN,NaN,NaN,0,8,2006,WD
2414,528218130,60,RL,58.0,17104,Pave,NaN,IR1,Lvl,AllPub,...,0,0,0,NaN,NaN,NaN,0,9,2006,New
1989,902207150,30,RM,60.0,8520,Pave,NaN,Reg,Lvl,AllPub,...,0,0,0,NaN,NaN,NaN,0,7,2007,WD
625,535105100,20,RL,NaN,9500,Pave,NaN,IR1,Lvl,AllPub,...,0,185,0,NaN,NaN,NaN,0,7,2009,WD


## Clean data in same method as training data

In [5]:
with open('../assets/columns.pkl', 'rb') as f:
    columns = pickle.load(f)

In [6]:
kaggle.columns = [col.lower().replace(' ', '_') for col in kaggle.columns]

In [7]:
kaggle.info();

<class 'pandas.core.frame.DataFrame'>
Int64Index: 879 entries, 2658 to 1939
Data columns (total 79 columns):
pid                879 non-null int64
ms_subclass        879 non-null int64
ms_zoning          879 non-null object
lot_frontage       719 non-null float64
lot_area           879 non-null int64
street             879 non-null object
alley              58 non-null object
lot_shape          879 non-null object
land_contour       879 non-null object
utilities          879 non-null object
lot_config         879 non-null object
land_slope         879 non-null object
neighborhood       879 non-null object
condition_1        879 non-null object
condition_2        879 non-null object
bldg_type          879 non-null object
house_style        879 non-null object
overall_qual       879 non-null int64
overall_cond       879 non-null int64
year_built         879 non-null int64
year_remod/add     879 non-null int64
roof_style         879 non-null object
roof_matl          879 non-null object
e

In [8]:
##Change overall_qual, overall_cond into objects
kaggle['overall_qual'] = kaggle['overall_qual'].astype(object)
kaggle['overall_cond'] = kaggle['overall_cond'].astype(object)

## Null values

For features that had null values that actually are supposed to be encoded as no such feature.

### Fence

In [9]:
kaggle['fence'].value_counts()

MnPrv    103
GdPrv     35
GdWo      32
MnWw       2
Name: fence, dtype: int64

In [10]:
##replace 'nan' which means no fence to 'NF'
kaggle['fence'] = kaggle['fence'].fillna('NF')

In [11]:
kaggle['fence'].value_counts()

NF       707
MnPrv    103
GdPrv     35
GdWo      32
MnWw       2
Name: fence, dtype: int64

### Misc_features

In [12]:
kaggle['misc_feature'].value_counts()

Shed    39
Gar2     1
Othr     1
Name: misc_feature, dtype: int64

In [13]:
kaggle['misc_feature'].unique()

array([nan, 'Shed', 'Othr', 'Gar2'], dtype=object)

In [14]:
kaggle['misc_feature'] = kaggle['misc_feature'].fillna('NoFeat')

In [15]:
kaggle[['misc_feature']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 879 entries, 2658 to 1939
Data columns (total 1 columns):
misc_feature    879 non-null object
dtypes: object(1)
memory usage: 13.7+ KB


### Pool Quality

In [16]:
kaggle['pool_qc'].value_counts()

Ex    3
TA    1
Name: pool_qc, dtype: int64

In [17]:
kaggle['pool_qc'].unique()

array([nan, 'Ex', 'TA'], dtype=object)

In [18]:
kaggle['pool_qc'] = kaggle['pool_qc'].fillna('NoPool')

In [19]:
kaggle[['pool_qc']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 879 entries, 2658 to 1939
Data columns (total 1 columns):
pool_qc    879 non-null object
dtypes: object(1)
memory usage: 13.7+ KB


### Masonry null values

In [20]:
##Veneer Type
kaggle['mas_vnr_type'].value_counts()

None       534
BrkFace    250
Stone       81
BrkCmn      12
CBlock       1
Name: mas_vnr_type, dtype: int64

In [21]:
kaggle['mas_vnr_type'].unique()

array(['None', 'BrkFace', 'Stone', 'BrkCmn', 'CBlock', nan], dtype=object)

In [22]:
kaggle['mas_vnr_type'].isnull().sum()

1

In [23]:
## Veneer Area
kaggle['mas_vnr_area'].isnull().sum()

1

In [24]:
kaggle.loc[:,['mas_vnr_type','mas_vnr_area']].isnull().sum()

mas_vnr_type    1
mas_vnr_area    1
dtype: int64

In [25]:
kaggle['mas_vnr_type'] = kaggle['mas_vnr_type'].fillna('None')

In [26]:
kaggle['mas_vnr_area'] = kaggle['mas_vnr_area'].fillna(0)

In [27]:
kaggle.loc[:,['mas_vnr_type','mas_vnr_area']].isnull().sum()

mas_vnr_type    0
mas_vnr_area    0
dtype: int64

### Alley

In [28]:
##replace 'nan' which means no valley access to 'no_valley_access'
kaggle['alley'] = kaggle['alley'].fillna('NVA')

In [29]:
kaggle['alley'].value_counts()

NVA     821
Grvl     35
Pave     23
Name: alley, dtype: int64

In [30]:
kaggle[['alley']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 879 entries, 2658 to 1939
Data columns (total 1 columns):
alley    879 non-null object
dtypes: object(1)
memory usage: 13.7+ KB


### Fireplace_qu

In [31]:
kaggle['fireplace_qu'].unique()

array([nan, 'Gd', 'Fa', 'TA', 'Po', 'Ex'], dtype=object)

In [32]:
kaggle['fireplace_qu'].value_counts()

Gd    221
TA    193
Fa     16
Po     15
Ex     12
Name: fireplace_qu, dtype: int64

In [33]:
kaggle['fireplace_qu'] = kaggle['fireplace_qu'].fillna('NoFireplace')

### Basement

In [34]:
##Define function to fill all NA data in basement columns with 'NB' (a.k.a 'No basement')
def replace_bsmt_na(df, bsmt_cols):
    for col in bsmt_columns:
        if df[col].dtype == 'object': 
            df[col].fillna('NB', inplace = True)

In [35]:
bsmt_columns = [col for col in kaggle.columns if 'bsmt' in col]

In [36]:
replace_bsmt_na(kaggle, bsmt_columns)

## Garage

In [37]:
##Define function to fill all NA data in basement columns with 'NB' (a.k.a 'No basement')
def replace_garage_na(df, garage_columns):
    for col in garage_columns:
        if df[col].dtype == 'object': 
            df[col].fillna('NG', inplace = True)

In [38]:
garage_columns = [col for col in kaggle.columns if 'garage' in col]

In [39]:
garage_columns

['garage_type',
 'garage_yr_blt',
 'garage_finish',
 'garage_cars',
 'garage_area',
 'garage_qual',
 'garage_cond']

In [40]:
garage_num_columns = [col for col in garage_columns if kaggle[col].dtype != 'object']

In [41]:
kaggle['garage_cars'].unique()

array([1, 2, 4, 0, 3])

In [42]:
kaggle['garage_area'].value_counts();

In [43]:
replace_garage_na(kaggle, garage_columns)

In [44]:
{col: kaggle[col].isnull().sum() for col in garage_columns}

{'garage_type': 0,
 'garage_yr_blt': 45,
 'garage_finish': 0,
 'garage_cars': 0,
 'garage_area': 0,
 'garage_qual': 0,
 'garage_cond': 0}

In [45]:
kaggle.drop('garage_cars', axis = 1, inplace = True)

In [46]:
kaggle.shape

(879, 78)

### Drop columns

In [47]:
kaggle.drop(['pid','lot_frontage', 'mas_vnr_type', 'mas_vnr_area', 'garage_yr_blt'], axis = 1, inplace = True)

In [48]:
kaggle.shape

(879, 73)

## Feature Engineering

In a typical house, we are usually interested in the total squared feet, not the two floor's individual squared feet.

In [49]:
kaggle['abvgrnd_sf'] = kaggle['1st_flr_sf'] + kaggle['2nd_flr_sf']

In [50]:
kaggle[['abvgrnd_sf','1st_flr_sf', '2nd_flr_sf']].head()

,abvgrnd_sf,1st_flr_sf,2nd_flr_sf
Id,,,
2658,1928,908,1020
2718,1967,1967,0
2414,1496,664,832
1989,968,968,0
625,1394,1394,0


In [51]:
kaggle.drop(['1st_flr_sf', '2nd_flr_sf'], axis = 1, inplace = True)

In [52]:
kaggle.shape

(879, 72)

### Dummy variable (entire dataset)

In [53]:
kaggle = pd.get_dummies(data = kaggle)

In [54]:
kaggle.shape

(879, 291)

In [55]:
kaggle.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 879 entries, 2658 to 1939
Columns: 291 entries, ms_subclass to sale_type_WD 
dtypes: int64(29), uint8(262)
memory usage: 430.9 KB


In [56]:
kaggle.head()

,ms_subclass,lot_area,year_built,year_remod/add,bsmtfin_sf_1,bsmtfin_sf_2,bsmt_unf_sf,total_bsmt_sf,low_qual_fin_sf,gr_liv_area,...,sale_type_COD,sale_type_CWD,sale_type_Con,sale_type_ConLD,sale_type_ConLI,sale_type_ConLw,sale_type_New,sale_type_Oth,sale_type_VWD,sale_type_WD
Id,,,,,,,,,,,,,,,,,,,,,
2658,190,9142,1910,1950,0,0,1020,1020,0,1928,...,0,0,0,0,0,0,0,0,0,1
2718,90,9662,1977,1977,0,0,1967,1967,0,1967,...,0,0,0,0,0,0,0,0,0,1
2414,60,17104,2006,2006,554,0,100,654,0,1496,...,0,0,0,0,0,0,1,0,0,0
1989,30,8520,1923,2006,0,0,968,968,0,968,...,0,0,0,0,0,0,0,0,0,1
625,20,9500,1963,1963,609,0,785,1394,0,1394,...,0,0,0,0,0,0,0,0,0,1


In [57]:
set(model_columns).difference(kaggle.columns) ##columns that don't appear in kaggle that are in my training

{'1st_flr_sf', '2nd_flr_sf', 'overall_cond', 'overall_qual'}

In [58]:
#insert missing levels
for missing_col in set(model_columns).difference(set(kaggle.columns)):
    kaggle[missing_col] = 0

In [59]:
set(model_columns).difference(kaggle.columns)

set()

In [60]:
set(kaggle.columns).difference(model_columns);

In [61]:
len(model_columns)

151

In [62]:
kaggle = kaggle[model_columns] ##no need to drop the above columns...

## Scale data

In [63]:
kaggle_sc = ss.transform(kaggle)

In [64]:
kaggle_sc.shape

(879, 151)

In [65]:
# kaggle_columns = set(kaggle.columns) training_columns = set(train.columns) new_columns = kaggle_columns.difference(training_columns) missing_columns = training_columns.difference(kaggle_columns) 

## Make predictions

In [66]:
preds = lasso.predict(kaggle_sc)

## Align predictions back with index and set up header

In [67]:
submission = pd.DataFrame(preds, index=kaggle.index, columns=['SalePrice'])

## Sort index (for proper submission)

In [68]:
submission.sort_index(inplace=True)

## Save to csv to submit

In [69]:
submission.to_csv('../datasets/numeric_only_lasso_12_6.csv')

In [70]:
submission.head()

,SalePrice
Id,
2,48705.488291
4,187742.789937
6,107587.149451
7,89931.192470
17,90922.519264


## Use command line `head` to check data is correct format

In [71]:
!head ../datasets/numeric_only_lr_lasso_12_6.csv

head: ../datasets/numeric_only_lr_lasso_12_6.csv: No such file or directory
